In [1]:
from experiments.mnli_utils import *
from tqdm import tqdm

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
task_tokenizer, task_model = create_tokenizer_and_model(
    "/export/home/Experiments/20200706/")

train_dataset, eval_dataset = create_datasets(tokenizer=task_tokenizer)

Params Trainable: 14768643
	bert.encoder.layer.10.attention.self.query.weight
	bert.encoder.layer.10.attention.self.query.bias
	bert.encoder.layer.10.attention.self.key.weight
	bert.encoder.layer.10.attention.self.key.bias
	bert.encoder.layer.10.attention.self.value.weight
	bert.encoder.layer.10.attention.self.value.bias
	bert.encoder.layer.10.attention.output.dense.weight
	bert.encoder.layer.10.attention.output.dense.bias
	bert.encoder.layer.10.attention.output.LayerNorm.weight
	bert.encoder.layer.10.attention.output.LayerNorm.bias
	bert.encoder.layer.10.intermediate.dense.weight
	bert.encoder.layer.10.intermediate.dense.bias
	bert.encoder.layer.10.output.dense.weight
	bert.encoder.layer.10.output.dense.bias
	bert.encoder.layer.10.output.LayerNorm.weight
	bert.encoder.layer.10.output.LayerNorm.bias
	bert.encoder.layer.11.attention.self.query.weight
	bert.encoder.layer.11.attention.self.query.bias
	bert.encoder.layer.11.attention.self.key.weight
	bert.encoder.layer.11.attention.self.ke

In [3]:
task_model.training, task_model.device

(False, device(type='cpu'))

In [4]:
from transformers import glue_output_modes, TrainingArguments, glue_compute_metrics

output_mode = glue_output_modes["mnli"]
def build_compute_metrics_fn(task_name: str):
    def compute_metrics_fn(p):
        if output_mode == "classification":
            preds = np.argmax(p.predictions, axis=1)
        elif output_mode == "regression":
            preds = np.squeeze(p.predictions)
        return glue_compute_metrics(task_name, preds, p.label_ids)

    return compute_metrics_fn


trainer = Trainer(
    model=task_model,
    args=TrainingArguments(output_dir="./tmp-output", per_device_train_batch_size=128, per_device_eval_batch_size=128, learning_rate=5e-5, logging_steps=100),
    data_collator=default_data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=build_compute_metrics_fn("mnli"),
)

train_batch_data_loader = get_dataloader(
    train_dataset,
    batch_size=160,
    data_collator=default_data_collator)

train_instance_data_loader = get_dataloader(
    train_dataset,
    batch_size=1,
    data_collator=default_data_collator)

eval_instance_data_loader = get_dataloader(
    eval_dataset,
    batch_size=1,
    data_collator=default_data_collator)

You are instantiating a Trainer but Tensorboard is not installed. You should consider installing it.


In [5]:
predictions = []
ground_truths = []
data_points = []
for i, test_inputs in enumerate(tqdm(train_batch_data_loader)):
    logits, labels, step_eval_loss = predict(
        trainer=trainer, model=task_model, inputs=test_inputs)

    ground_truths.extend(labels.tolist())
    predictions.extend(logits.argmax(axis=1).tolist())
    data_points.extend([
        x.squeeze(dim=0)
        for x in torch.split(test_inputs["input_ids"],
                             split_size_or_sections=1, dim=0)])

100%|██████████| 2455/2455 [13:27<00:00,  3.04it/s]


In [6]:
with open("/export/home/Data/Glue/MNLI/train.tsv") as f:
    original_lines = [d.strip().split("\t") for d in f.readlines()]

with open("/export/home/Experiments/20200718/train.tsv") as f:
    modified_lines = [d.strip().split("\t") for d in f.readlines()]

In [7]:
# https://github.com/huggingface/transformers/blob/master/src/transformers/data/processors/glue.py#L238
original_labels = [l[-1] for l in original_lines]
modified_labels = [l[-1] for l in modified_lines]

In [8]:
original_labels[1:] == [train_dataset.label_list[l] for l in ground_truths]

True

In [9]:
modified_labels[1:] == [train_dataset.label_list[l] for l in ground_truths]

False

In [10]:
original_labels[1:] == [train_dataset.label_list[l] for l in predictions]

False

In [11]:
modified_labels[1:] == [train_dataset.label_list[l] for l in predictions]

True

In [44]:
original_lines[1:][252321]

['252321',
 '126359',
 '126359n',
 'government',
 '( ( The ( final rule ) ) ( was ( determined ( to ( be ( ( an ( ( economically significant ) ( regulatory action ) ) ) ( under ( Executive Order ) ) ) ) ) ) ) )',
 '( ( The ( final rule ) ) ( ( ( ( was ( economically significant ) ) and ) ( ( was impressive ) ( reading ( for students ) ) ) ) . ) )',
 '(ROOT (S (NP (DT The) (JJ final) (NN rule)) (VP (VBD was) (VP (VBN determined) (S (VP (TO to) (VP (VB be) (NP (NP (DT an) (ADJP (RB economically) (JJ significant)) (JJ regulatory) (NN action)) (PP (IN under) (NP (NNP Executive) (NNP Order)))))))))))',
 '(ROOT (S (NP (DT The) (JJ final) (NN rule)) (VP (VP (VBD was) (ADJP (RB economically) (JJ significant))) (CC and) (VP (VBD was) (ADJP (JJ impressive)) (S (VP (VBG reading) (PP (IN for) (NP (NNS students))))))) (. .)))',
 'The final rule was determined to be an economically significant regulatory action under Executive Order',
 'The final rule was economically significant and was impressive 

In [45]:
task_tokenizer.decode(data_points[252321])

'[CLS] The final rule was determined to be an economically significant regulatory action under Executive Order [SEP] The final rule was economically significant and was impressive reading for students. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'